In [9]:
import pandas as pd
import os
import glob
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
import mysql.connector
import numpy as np

In [11]:
def load_cake_data():
    # Establish a connection to the database
    connection = mysql.connector.connect(
        host='77.68.29.139',
        user='leadsense_pbi',
        password='qLi6f6^7Bgw95l4%',
        database=' leadsense_pbi'
    )

    # Create a cursor object to interact with the database
    cursor = connection.cursor()

    # Execute the query
    query = """SELECT lead_ref as 'Lead ID', 
    client_name as 'Client',
    cell_phone as 'Cell Phone',
    date_sold as 'Date Sold',
    aff_name as 'Affiliate',
    keyword as 'Keyword',
    userage as 'Age',
    matchtype as 'Match Type',
    smoker as 'Smoker',
    postcode as 'Postcode',
    device as 'Device',
    gender as 'Gender',
    track_device as 'Track Device',
    browser as 'Browser', 
    current_insurance as 'Current Insurance',
    campaignid as 'Campaign ID',
    lead_url as 'Lead URL'
    FROM leadsense_pbi_all_data_v2 WHERE client_name IN ('Usay','Heath Crawford') and channel_alias='PPC' ORDER BY date_sold;"""

    cursor.execute(query)

    # Fetch all the results
    results = cursor.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cursor.description]

    # Create a DataFrame from the results
    lead_data = pd.DataFrame(results, columns=column_names)


    # Close the cursor and the connection
    cursor.close()
    connection.close()
    
    return(lead_data)

cake_data = load_cake_data()
cake_data.head()

,Lead ID,Client,Cell Phone,Date Sold,Affiliate,Keyword,Age,Match Type,Smoker,Postcode,Device,Gender,Track Device,Browser,Current Insurance,Campaign ID,Lead URL
0,5402D7B2,Heath Crawford,Landline,2022-01-04 08:08:51,Internal Google,health insurance,54,e,No,SG8 0LP,c,Male,Desktop,Safari,Yes Private,30,https://www.bestcompare.com/health-insurance/q...
1,A99E4C29,Heath Crawford,Mobile,2022-01-04 08:25:37,Internal Google,private healthcare,62,e,No,PE4 5DX,c,Male,Desktop,Chrome,Yes Company,30,https://www.bestcompare.com/health-insurance/q...
2,A728DE15,Heath Crawford,Mobile,2022-01-04 08:35:25,Internal Google,private medical insurance,52,e,No,BH21 4DD,c,Male,Desktop,Chrome,Yes Private,30,https://www.bestcompare.com/health-insurance/q...
3,D55005E6,Heath Crawford,Mobile,2022-01-04 08:49:12,Internal Google,health insurance,34,e,No,CH42 9NU,m,Female,Smartphone,Mobile Safari,Yes Private,30,https://www.bestcompare.com/health-insurance/q...
4,D3F0EAE6,Heath Crawford,Mobile,2022-01-04 10:00:19,Internal Google,personal health insurance,66,e,No,CM15 8BU,c,Female,,,Yes Company,30,https://www.bestcompare.com/health-insurance/q...


In [12]:
# Extract date features

cake_data['Date Sold'] = pd.to_datetime(cake_data['Date Sold'])
cake_data['day_of_week'] = cake_data['Date Sold'].dt.dayofweek
cake_data['hour_of_day'] = cake_data['Date Sold'].dt.hour

#Add post code data

postcode = pd.read_csv('postcodes.csv')

cake_data['Postcode'] = cake_data['Postcode'].str.replace(' ', '').str.lower()
cake_data = cake_data.merge(postcode,on='Postcode',how='left')


# Add age bins
# Define the bin edges
bin_edges = [18, 24, 35, 44, 55, 65,  np.inf]  # Example bin edges: 0-30, 30-50, 50 and above
# Create the bin labels
bin_labels = ['18-24', '25-34', '35-44', '45-54', '55-54','65<']
# Use pandas.cut to bin the 'age' variable
cake_data['Age_bin'] = pd.cut(cake_data['Age'], bins=bin_edges, labels=bin_labels, right=False)
cake_data.head()


,Lead ID,Client,Cell Phone,Date Sold,Affiliate,Keyword,Age,Match Type,Smoker,Postcode,...,Gender,Track Device,Browser,Current Insurance,Campaign ID,Lead URL,day_of_week,hour_of_day,Region,Age_bin
0,5402D7B2,Heath Crawford,Landline,2022-01-04 08:08:51,Internal Google,health insurance,54,e,No,sg80lp,...,Male,Desktop,Safari,Yes Private,30,https://www.bestcompare.com/health-insurance/q...,1,8,Eastern,45-54
1,A99E4C29,Heath Crawford,Mobile,2022-01-04 08:25:37,Internal Google,private healthcare,62,e,No,pe45dx,...,Male,Desktop,Chrome,Yes Company,30,https://www.bestcompare.com/health-insurance/q...,1,8,Eastern,55-54
2,A728DE15,Heath Crawford,Mobile,2022-01-04 08:35:25,Internal Google,private medical insurance,52,e,No,bh214dd,...,Male,Desktop,Chrome,Yes Private,30,https://www.bestcompare.com/health-insurance/q...,1,8,South West,45-54
3,D55005E6,Heath Crawford,Mobile,2022-01-04 08:49:12,Internal Google,health insurance,34,e,No,ch429nu,...,Female,Smartphone,Mobile Safari,Yes Private,30,https://www.bestcompare.com/health-insurance/q...,1,8,North West,25-34
4,D3F0EAE6,Heath Crawford,Mobile,2022-01-04 10:00:19,Internal Google,personal health insurance,66,e,No,cm158bu,...,Female,,,Yes Company,30,https://www.bestcompare.com/health-insurance/q...,1,10,Eastern,65<


In [59]:
# Load client data


folder_path = "G:/Shared drives/Support Ops   Quality Management/Quality Management/Datasets & Reports  Client/Usay/RawClientData/PPC"  # Replace with the actual folder path

# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]

# Initialize an empty DataFrame to store the combined data
usay_client = pd.DataFrame()

# Iterate over each CSV file and append its data to the combined DataFrame
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    
    df = pd.read_csv(file_path)
    df.drop('Status',inplace=True,axis=1)
    cols = ['Internal ID','Revenue','Status.1']
    df = df[cols]
    
    # keep last duplicate columns
    usay_client = usay_client.append(df, ignore_index=True)

# Load client data

usay_client.columns = ['Lead ID','Premium','Status']
usay_client = usay_client[['Lead ID', 'Status', 'Premium']]

hc_path = "G:/Shared drives/Support Ops   Quality Management/Quality Management/Datasets & Reports  Client/Heath Crawford/RawClientData/Health/Heath_Crawford_20230102_20230904.csv"  # Replace with the actual folder path
hc_client_data = pd.read_csv(hc_path,usecols  = ['Lead Reference Number ','Status','Annual Premium: '])
hc_client_data.columns = ['Lead ID','Status','Premium']
hc_client_data.head()

client_data = pd.concat([usay_client,hc_client_data], axis=0)
client_data.head()


C:\Users\asus\AppData\Local\Temp\ipykernel_5880\2776371380.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  usay_client = usay_client.append(df, ignore_index=True)
C:\Users\asus\AppData\Local\Temp\ipykernel_5880\2776371380.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  usay_client = usay_client.append(df, ignore_index=True)
C:\Users\asus\AppData\Local\Temp\ipykernel_5880\2776371380.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  usay_client = usay_client.append(df, ignore_index=True)
C:\Users\asus\AppData\Local\Temp\ipykernel_5880\2776371380.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  usay_client = usay_client.ap

,Lead ID,Status,Premium
0,151BF8CD,NOT_INTERESTED,NaN
1,75BD6592,NOT_INTERESTED,NaN
2,7B854E93,NOT_INTERESTED,NaN
3,E4112DB0,NEW,NaN
4,E30501CD,NOT_INTERESTED,NaN


In [63]:
cake_client_merged = pd.merge(cake_data,client_data,on='Lead ID',how='left')
cake_client_merged.head()

,Lead ID,Client,Cell Phone,Date Sold,Affiliate,Keyword,Age,Match Type,Smoker,Postcode,...,Browser,Current Insurance,Campaign ID,Lead URL,day_of_week,hour_of_day,Region,Age_bin,Status,Premium
0,5402D7B2,Heath Crawford,Landline,2022-01-04 08:08:51,Internal Google,health insurance,54,e,No,sg80lp,...,Safari,Yes Private,30,https://www.bestcompare.com/health-insurance/q...,1,8,Eastern,45-54,NaN,NaN
1,A99E4C29,Heath Crawford,Mobile,2022-01-04 08:25:37,Internal Google,private healthcare,62,e,No,pe45dx,...,Chrome,Yes Company,30,https://www.bestcompare.com/health-insurance/q...,1,8,Eastern,55-54,NaN,NaN
2,A728DE15,Heath Crawford,Mobile,2022-01-04 08:35:25,Internal Google,private medical insurance,52,e,No,bh214dd,...,Chrome,Yes Private,30,https://www.bestcompare.com/health-insurance/q...,1,8,South West,45-54,NaN,NaN
3,D55005E6,Heath Crawford,Mobile,2022-01-04 08:49:12,Internal Google,health insurance,34,e,No,ch429nu,...,Mobile Safari,Yes Private,30,https://www.bestcompare.com/health-insurance/q...,1,8,North West,25-34,NaN,NaN
4,D3F0EAE6,Heath Crawford,Mobile,2022-01-04 10:00:19,Internal Google,personal health insurance,66,e,No,cm158bu,...,,Yes Company,30,https://www.bestcompare.com/health-insurance/q...,1,10,Eastern,65<,NaN,NaN


In [113]:
cake_client_usay = cake_client_merged.loc[cake_client_merged['Client']=='Usay',:]

# Define your date range
current_start_date = '2023-07-01'
current_end_date = '2023-07-30'

# Define your date range
comparison_start_date = '2023-05-01'
comparison_end_date = '2023-06-30'

# Use .loc[] to filter rows based on the date range
current_data_usay = cake_client_usay.loc[(cake_client_usay['Date Sold'] >= start_date) & (cake_client_usay['Date Sold'] <= end_date)]
comparison_data_usay = cake_client_usay.loc[(cake_client_usay['Date Sold'] >= comparison_start_date) & (cake_client_usay['Date Sold'] <= comparison_end_date)]
comparison_data_usay.head()


,Lead ID,Client,Cell Phone,Date Sold,Affiliate,Keyword,Age,Match Type,Smoker,Postcode,...,Browser,Current Insurance,Campaign ID,Lead URL,day_of_week,hour_of_day,Region,Age_bin,Status,Premium
18125,F057720A,Usay,Mobile,2023-05-01 07:17:40,Internal Google,bupa health insurance price,36,e,No,sw35tt,...,Chrome Mobile iOS,No,30,https://www.bestcompare.com/health-insurance/q...,0,7,London,35-44,NOT_INTERESTED,NaN
18126,4A579F4E,Usay,Mobile,2023-05-01 07:20:58,Innovativity,+private +insurance +uk,21,b,No,wn86dh,...,Mobile Safari,No,6258,https://www.mymedicalinsurance.co.uk/private/,0,7,North West,18-24,QUOTE,NaN
18127,2C9B98CB,Usay,Mobile,2023-05-01 07:26:14,Internal Google,compare individual health plans,38,e,No,de119sq,...,Chrome Mobile,No,30,https://www.bestcompare.com/health-insurance/p...,0,7,East Midlands,35-44,NOT_INTERESTED,NaN
18128,D5805677,Usay,Mobile,2023-05-01 07:26:34,Internal Google,bupa health insurance,46,e,No,cb250dn,...,Mobile Safari,No,30,https://www.bestcompare.com/health-insurance/p...,0,7,Eastern,45-54,INVALID_PHONE,NaN
18129,3C9A4657,Usay,Mobile,2023-05-01 07:31:04,Innovativity,+health +insurance +policy,53,b,No,cf314sn,...,Chrome Mobile,No,6258,https://www.mymedicalinsurance.co.uk/policy/,0,7,Wales,45-54,NOT_INTERESTED,NaN


In [65]:
def compare_distribution(column,curret_data,comparison_data):
    
    # Calculate the proportions of each age category in both DataFrames
    prop_comparison = comparison_data[column].value_counts(normalize=True).sort_index()
    prop_current = curret_data[column].value_counts(normalize=True).sort_index()

    # Create a DataFrame to store the differences in proportions
    categories = sorted(set(prop_comparison.index) | set(prop_current.index))
    prop_diff_df = pd.DataFrame({'Category': categories,
                                 'Proportion Difference (%)': ((prop_comparison - prop_current) * 100).fillna(0)})

    # Display the resulting DataFrame
    return(prop_diff_df)
age_diff =  compare_distribution('Age_bin',current_data_usay,comparison_data_usay)  
switch_diff =  compare_distribution('Current Insurance',current_data_usay,comparison_data_usay) 
region_diff =  compare_distribution('Region',current_data_usay,comparison_data_usay)



# Display the resulting DataFrame
print(region_diff)


                                          Category  Proportion Difference (%)
East Midlands                        East Midlands                  -0.856586
Eastern                                    Eastern                  -1.255923
London                                      London                   0.870040
North East                              North East                   1.556011
North West                              North West                   0.897923
Northern Ireland                  Northern Ireland                   0.399922
Scotland                                  Scotland                   0.031393
South East                              South East                   0.920542
South West                              South West                   0.676026
Wales                                        Wales                  -0.384713
West Midlands                        West Midlands                  -2.055182
Yorkshire and The Humber  Yorkshire and The Humber              

In [81]:
import pandas as pd

def compare_distribution_bivariate(column, current_data, comparison_data):
    # Calculate the mean 'Premium' for each category in both DataFrames
    prop_comparison = comparison_data.groupby(column)['Premium'].mean().sort_index()
    prop_current = current_data.groupby(column)['Premium'].mean().sort_index()

    # Create a DataFrame to store the differences in means
    categories = sorted(set(prop_comparison.index) | set(prop_current.index))
    prop_diff_df = pd.DataFrame({'Category': categories,
                                 'Mean Difference': prop_comparison - prop_current})

    # Display the resulting DataFrame
    return prop_diff_df

# Example usage
print('Bivariate comparison')
age_diff = compare_distribution_bivariate('Region', current_data_usay, comparison_data_usay)
print(age_diff)


Bivariate comparison
                                          Category  Mean Difference
Region                                                             
East Midlands                        East Midlands      1452.570000
Eastern                                    Eastern      -371.836364
London                                      London       622.467273
North East                              North East              NaN
North West                              North West       -36.572667
Northern Ireland                  Northern Ireland        79.061667
Scotland                                  Scotland      -598.860000
South East                              South East      -931.524000
South West                              South West      -190.490000
Wales                                        Wales              NaN
West Midlands                        West Midlands      -110.715000
Yorkshire and The Humber  Yorkshire and The Humber       -22.060000


In [76]:
correlation = current_data_usay.corr()
correlation

,Age,Campaign ID,day_of_week,hour_of_day,Premium
Age,1.000000,0.364870,0.017483,-0.114785,0.583215
Campaign ID,0.364870,1.000000,-0.010258,-0.144593,0.066141
day_of_week,0.017483,-0.010258,1.000000,-0.101758,0.086536
hour_of_day,-0.114785,-0.144593,-0.101758,1.000000,0.175774
Premium,0.583215,0.066141,0.086536,0.175774,1.000000


In [77]:
cake_client_hc = cake_client_merged.loc[cake_client_merged['Client']=='Heath Crawford',:]

# Define your date range
current_start_date = '2023-07-01'
current_end_date = '2023-07-30'

# Define your date range
comparison_start_date = '2023-05-01'
comparison_end_date = '2023-06-30'

# Use .loc[] to filter rows based on the date range
current_data_hc = cake_client_hc.loc[(cake_client_hc['Date Sold'] >= start_date) & (cake_client_hc['Date Sold'] <= end_date)]
comparison_data_hc = cake_client_hc.loc[(cake_client_hc['Date Sold'] >= comparison_start_date) & (cake_client_hc['Date Sold'] <= comparison_end_date)]
comparison_data_hc.head()

,Lead ID,Client,Cell Phone,Date Sold,Affiliate,Keyword,Age,Match Type,Smoker,Postcode,...,Browser,Current Insurance,Campaign ID,Lead URL,day_of_week,hour_of_day,Region,Age_bin,Status,Premium
18173,FB4B5D50,Heath Crawford,Landline,2023-05-02 08:08:27,Innovativity,private medical insurance,66,e,No,bs311jh,...,Chrome Mobile,Yes Private,6258,https://www.mymedicalinsurance.co.uk/private/,1,8,South West,65<,DNC - Too expensive,0.0
18175,AB9BBF55,Heath Crawford,Mobile,2023-05-02 08:38:08,Innovativity,+health +insurance +policy,66,b,No,sa148ts,...,Chrome Mobile,Yes Private,6258,https://www.mymedicalinsurance.co.uk/policy/,1,8,Wales,65<,DNQ - travel,0.0
18178,5D63A32A,Heath Crawford,Landline,2023-05-02 09:17:22,Innovativity,medical insurance,65,p,No,sk224hz,...,Chrome Mobile,Yes Private,6258,https://www.mymedicalinsurance.co.uk/over-50s/,1,9,East Midlands,65<,DNC,0.0
18179,71FD38AD,Heath Crawford,Mobile,2023-05-02 09:19:46,Internal Google,best health insurance plan,36,e,No,nn144jq,...,Chrome Mobile,No,30,https://www.bestcompare.com/health-insurance/q...,1,9,East Midlands,35-44,Contact - Load Back Into Dialler,500.0
18180,5B786249,Heath Crawford,Mobile,2023-05-02 09:24:44,Internal Google,private health care,36,e,No,ip294uw,...,Google,No,30,https://www.bestcompare.com/health-insurance/f...,1,9,Eastern,35-44,DNC,0.0


In [83]:
# Compare clients

import pandas as pd

def compare_distribution_clients(column, client1_current, client2_current):
    # Calculate the mean 'Premium' for each category in both DataFrames
    prop_comparison = client1_current.groupby(column)['Premium'].mean().sort_index()
    prop_current = client2_current.groupby(column)['Premium'].mean().sort_index()

    # Create a DataFrame to store the differences in means
    categories = sorted(set(prop_comparison.index) | set(prop_current.index))
    prop_diff_df = pd.DataFrame({'Category': categories,
                                 'Mean Difference': prop_comparison - prop_current})

    # Display the resulting DataFrame
    return prop_diff_df

# Example usage
print('Client variation')
age_diff = compare_distribution_clients('Age_bin',current_data_hc,current_data_usay)
print(age_diff)


Client variation
        Category  Mean Difference
Age_bin                          
18-24      18-24      -580.160000
25-34      25-34      -456.292410
35-44      35-44      -995.142150
45-54      45-54     -1198.053086
55-54      55-54      -911.801465
65<          65<     -1763.645820


In [97]:
# Potential upflift bivariaate

current_data_usay[['Age_bin','Premium']].groupby('Age_bin').count()
current_data_usay[['Age_bin','Premium']].groupby('Age_bin').mean()

# Calculate the count of each group
group_counts = current_data_usay[['Age_bin','Premium']].groupby('Age_bin').count()

# Calculate the percentage of the total for each group
group_counts['Percentage of Total'] = (group_counts['Premium']*100)/sum(group_counts['Premium'])
# Calculate the percentage of the total for each group
group_counts['Premium'] =  current_data_usay[['Age_bin','Premium']].groupby('Age_bin').mean()

# Display the DataFrame with percentages
group_counts.reset_index()

,Age_bin,Premium,Percentage of Total
0,18-24,580.160000,5.084746
1,25-34,656.796000,8.474576
2,35-44,1164.882857,23.728814
3,45-54,1607.945455,18.644068
4,55-54,1440.584000,25.423729
5,65<,2488.603636,18.644068


In [99]:
import pandas as pd

# Sample DataFrame (replace with your actual DataFrame)
data = {
    'Age_bin': ['18-24', '25-34', '35-44', '45-54', '55-54', '65<'],
    'Premium': [580.16, 656.796, 1164.882857, 1607.945455, 1440.584, 2488.603636],
    'Percentage of Total': [5.084746, 8.474576, 23.728814, 18.644068, 25.423729, 18.644068]
}

df = pd.DataFrame(data)

# Find the original average premium for '65<'
original_average_premium_65 = df.loc[df['Age_bin'] == '65<', 'Premium'].values[0]

# Increase the percentage by 5%
increased_percentage = df.loc[df['Age_bin'] == '65<', 'Percentage of Total'].values[0] + 5

# Calculate the adjusted average premium for '65<' based on the increased percentage
adjusted_average_premium_65 = original_average_premium_65 * (1 + (increased_percentage / 100))

# Calculate the overall uplift in Premium by adjusting the '65<' category
overall_uplift_premium = (adjusted_average_premium_65 - original_average_premium_65) * len(df[df['Age_bin'] == '65<'])

# Update the DataFrame with the increased percentage for '65<' (if needed)
df.loc[df['Age_bin'] == '65<', 'Percentage of Total'] = increased_percentage

# Display the updated DataFrame, the adjusted average premium for '65<', and the overall uplift in Premium
print(df)
print(f'Adjusted Average Premium for "65<": {adjusted_average_premium_65:.2f}')
print(f'Overall Uplift in Premium: {overall_uplift_premium:.2f}')


  Age_bin      Premium  Percentage of Total
0   18-24   580.160000             5.084746
1   25-34   656.796000             8.474576
2   35-44  1164.882857            23.728814
3   45-54  1607.945455            18.644068
4   55-54  1440.584000            25.423729
5     65<  2488.603636            23.644068
Adjusted Average Premium for "65<": 3077.01
Overall Uplift in Premium: 588.41


In [115]:
# Client leads exchange uplift calculation

grouped_hc = current_data_hc[['Premium','Current Insurance']].groupby('Current Insurance').mean()
grouped_hc['Percentages'] = (current_data_hc[['Premium','Current Insurance']].groupby('Current Insurance').count()*100)/current_data_hc.shape[0]
grouped_hc

grouped_usay = current_data_usay[['Premium','Current Insurance']].groupby('Current Insurance').mean()
grouped_usay['Percentages'] = current_data_usay[['Premium','Current Insurance']].groupby('Current Insurance').count()
grouped_usay

,Premium,Percentages
Current Insurance,,
No,1491.503333,54
Yes Company,1171.440000,1
Yes Private,1572.750000,4


In [50]:
import smtplib
from email.mime.text import MIMEText

smtp_server = 'http://smtppro.zoho.eu/'
smtp_port = 465
smtp_username = 'pranav@leadsense.com'
smtp_password = 'xxxxx@1994'

subject = 'Subject of your email'
body = 'This is the body of your email.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = smtp_username
msg['To'] = 'pranav@leadsense.com'  # Replace with the recipient's email address


try:
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)
    server.sendmail(smtp_username, msg['To'], msg.as_string())
    server.quit()
    print('Email sent successfully!')
except Exception as e:
    print('Email could not be sent. Error:', str(e))


Email could not be sent. Error: [Errno 11001] getaddrinfo failed
